In [1]:
# Libraries needed for WebCam Face_Detection.
import cv2
import os
import numpy as np
import torch
from PIL import Image
from facenet_pytorch import InceptionResnetV1,MTCNN
import sqlite3
from scipy.spatial.distance import euclidean
import torch.nn.functional as F

In [2]:
os.getcwd()

'C:\\Users\\yeswa'

In [3]:
os.chdir("C:/Users/yeswa/OneDrive/Pictures/Face_Recognization_Photos")

In [4]:
#below variables are FaceNet related-> FaceNet creates Embedding Vectors of each face detected by MTCNN model.
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')# This makes the enviroment use GPU if exist other wise CPU because of performance.
facenet = InceptionResnetV1(pretrained="vggface2").eval().to(device)# creating object for FaceNet Model.
mtcnn = MTCNN(keep_all=True,device=device)# Creating object for MTCNN model.

In [5]:
folder_path = "C:/Users/yeswa/OneDrive/Pictures/Face_Recognization_Photos"
data = {}
def get_known_embedding(path):
    for folder_path in os.listdir(path):
        images_path = os.path.join(path,folder_path)
        if not os.path.isdir(images_path):
            continue
        embeddings = []
        for image in os.listdir(images_path):
            img_path = os.path.join(images_path,image)
            bgr_images = cv2.imread(img_path)
            if bgr_images is None:
                continue
            rgb_faces = cv2.cvtColor(bgr_images,cv2.COLOR_BGR2RGB)
            img_pil = Image.fromarray(rgb_faces)
            detect_faces = mtcnn(img_pil)
            if detect_faces is None:
                print("No Face is Detected:",{images_path})
                continue
            if detect_faces.ndim==3:
                batch_tensor = detect_faces.unsqueeze(0).to(device)
            else:
                batch_tensor = detect_faces.to(device)
            embedding = facenet(batch_tensor).detach().cpu().squeeze(0)
            embeddings.append(embedding)
        if embeddings:
            data[folder_path] = embeddings
    return data

In [6]:
known_faces = get_known_embedding(folder_path)
known_faces

{'Akhil': [tensor([ 0.0610, -0.0361, -0.0044,  0.0261,  0.0122,  0.0311,  0.0330,  0.0058,
          -0.0468,  0.0077,  0.0076, -0.0012, -0.0690, -0.0079,  0.0028,  0.0240,
          -0.0018,  0.0770, -0.0564,  0.0375,  0.0093,  0.0315,  0.0415, -0.0702,
          -0.0145,  0.0232,  0.0400, -0.0392,  0.0155, -0.1033,  0.0444,  0.0235,
          -0.0151,  0.0539,  0.0367,  0.0280, -0.0004, -0.0266, -0.0209, -0.0119,
           0.0536, -0.0967, -0.0248, -0.0478, -0.0279, -0.0420, -0.0451, -0.0196,
          -0.0018,  0.0438,  0.0671, -0.0231,  0.0175,  0.0327, -0.0239, -0.0302,
          -0.0059, -0.0325, -0.0005,  0.0269,  0.0045,  0.0061, -0.0034,  0.0915,
          -0.0161, -0.0557,  0.0256, -0.0442,  0.0707, -0.0662, -0.0717, -0.0173,
           0.0076, -0.0279,  0.0124,  0.0016,  0.0075, -0.0588,  0.0105, -0.0185,
          -0.0062, -0.0420, -0.0414, -0.0392,  0.0315,  0.0673,  0.0469, -0.0141,
          -0.0224,  0.0405, -0.0565, -0.0257, -0.0221, -0.0010, -0.0115,  0.0168,
       

In [7]:
# below code is read frames of webcam
cap = cv2.VideoCapture(0) # initate the webcam
while True: # if flow is true then go inside the loop
    ret,frame = cap.read() # ret->True/Fasle of VideoCapturing if capturing True not capturing False
    if not ret: #condition on frames are getting or not.
        break # if condition true then break flow here itself.
    boxes,prob = mtcnn.detect(frame)# Detects faces from CC Camera and shows green box around face->box,prob->indicates accuracy of face >=95 then it is face.
    if boxes is not None:
        for box in boxes:
            x1,y1,x2,y2 = [int(coord) for coord in box]
            cv2.rectangle(frame,(x1,y1),(x2,y2),(0,255,0),2)
            # Above loop and two lines are going to display green line around the face lively.
            face = frame[y1:y2,x1:x2]# This variable consist of that face numerical vector
            face_tensor = mtcnn(frame)
            if face_tensor is None:
                continue
            if len(face_tensor.shape) == 3:
                batch_face = face_tensor.unsqueeze(0).to(device)
            else:
                batch_face = face_tensor.to(device)
            live_embeddings = facenet(batch_face).detach().cpu().squeeze(0)
            for person_name,embeddings in known_faces.items():
                for emb in embeddings:
                    if live_embeddings.shape != emb.shape:
                        continue
                    distance = euclidean(live_embeddings.numpy().flatten(),emb.numpy().flatten())
            if distance<0.8:
                print("Known person")
            elif distance>=1.0:
               print("unknown person")
    cv2.imshow("Frame",frame)# Display the image from vectors.
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
cap.release()
cv2.destroyAllWindows()

unknown person
unknown person
unknown person
unknown person
unknown person
unknown person
unknown person
unknown person
unknown person
unknown person
unknown person
unknown person
unknown person
unknown person
unknown person
unknown person
unknown person
unknown person
unknown person
unknown person
unknown person
unknown person
unknown person
unknown person
unknown person
unknown person
unknown person
unknown person
unknown person
unknown person
unknown person
unknown person
unknown person
unknown person
unknown person
unknown person
unknown person
unknown person
unknown person
unknown person
unknown person
unknown person
unknown person
unknown person
unknown person
unknown person
Known person
unknown person
unknown person
Known person
Known person
Known person
Known person
Known person
Known person
Known person
Known person
Known person
Known person
Known person
Known person
Known person
Known person
Known person
Known person
Known person
Known person
Known person
Known person
Known p